In [1]:
from models.builers.retriever import Retriever
from data.phisherman import LoadPhishingDataset

In [2]:
# Load dataset
dataset = LoadPhishingDataset()

Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail from {'Email Text': nan, 'Email Type': 'Phishing Email'}
Could not create PhishingEmail f

In [4]:
from models.model_loader_helpers import create_models

models_to_create = {"TF-IDF": {},
                    "BM25": {}}

create_models(documents=dataset, dataset_name="Phisher", models=models_to_create, save=True)

Creating TF-IDF model
GetCorpusVocabulary Elapsed: 2.678410053253174s
GetInverseDocumentFrequencies Elapsed: 3.7538979053497314s
GetDocumentsTFIDFVectors Elapsed: 29.269556999206543s
Saving model 'TF-IDF' at: models/pickled_models/Phisher/TF-IDF.pickle
Creating BM25 model
GetCorpusVocabulary Elapsed: 2.5160582065582275s
GetInverseDocumentFrequencies Elapsed: 3.8616950511932373s
GetDocumentLengths Elapsed: 1.2464590072631836s
GetDocumentBM25Vectors Elapsed: 35.76833415031433s
Saving model 'BM25' at: models/pickled_models/Phisher/BM25.pickle


{'TF-IDF': <models.TFIDF.TFIDF at 0x120ae2230>,
 'BM25': <models.BM25.BM25 at 0x120ae33a0>}

In [12]:
from models.model_loader_helpers import load_models

models = load_models("Phisher", models_to_create)

In [7]:
from data.query import Query

queries = []

for rel in dataset:
    id = rel.Id
    queries.append(Query(text=rel.text, id=id, relevant_document_ids=dataset.getRelatedDocuments(rel)))

('the other side of * galicismos * * galicismo * is a spanish term which names the improper introduction of french words which are spanish sounding and thus very deceptive to the ear . * galicismo * is often considered to be a * barbarismo * . what would be the term which designates the opposite phenomenon , that is unlawful words of spanish origin which may have crept into french ? can someone provide examples ? thank you joseph m kozono < kozonoj @ gunet . georgetown . edu >',
 11321)

In [ ]:
queries[1].GetQuery(), queries[0].GetNumberOfRelevantDocuments()

In [6]:
def retrieveQueryAndGetScore(model: Retriever, query: Query, k: int):
    retrieved_documents = model.Lookup(query=query.GetQuery(), k=k)
    relevancies = []
    for document in retrieved_documents:
        if query.IsDocumentRelevant(document):
            relevancies.append(True)
        else:
            relevancies.append(False)
    return relevancies

In [8]:
def reciprocalRank(relevancies):
    for i, relevancy in enumerate(relevancies):
        if relevancy:
            return 1/(i+1)
    return 0

def meanReciprocalRank(scores):
    reciprocal_ranks = []
    for score in scores:
        reciprocal_ranks.append(reciprocalRank(score))

    return sum(reciprocal_ranks) / len(reciprocal_ranks)

def precision(relevancies):
    return sum([1 if relevancy else 0 for relevancy in relevancies]) / len(relevancies)

def recall(relevancies, query: Query):
    return sum([1 if relevancy else 0 for relevancy in relevancies]) / min(len(relevancies), query.GetNumberOfRelevantDocuments())

def calculate_metrics(results):
    """ Calculates precision, recall and accuracy based on the results of a query. True or false values are used to indicate whether a document is relevant or not.
    
    Args:
        results (list): A list of lists of booleans, where each list of booleans represents the retrieved documents for a query

    Returns:
        precision (float): Precision of the results
        recall (float): Recall of the results
        accuracy (float): Accuracy of the results
    """
    total_true_positives = 0  # Relevant documents correctly retrieved
    total_false_positives = 0 # Non-relevant documents incorrectly retrieved
    total_false_negatives = 0 # Relevant documents missed
    total_true_negatives = 0  # Non-relevant documents correctly not retrieved

    for query_results in results:
        true_positives = sum(query_results)
        false_positives = len(query_results) - true_positives
        # Assuming the length of the list is the total number of relevant documents for the query
        false_negatives = len(query_results) - true_positives  
        # True negatives can't be calculated without knowing the total number of non-relevant documents

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

    # Calculating metrics
    precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
    recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0
    accuracy = (total_true_positives + total_true_negatives) / (total_true_positives + total_false_positives + total_false_negatives + total_true_negatives) if (total_true_positives + total_false_positives + total_false_negatives + total_true_negatives) > 0 else 0

    mrr = meanReciprocalRank(results)

    return precision, recall, accuracy, mrr

In [9]:
import time

def timeFunction(function, **args):
    time_before = time.perf_counter()
    output = function(**args)
    time_after = time.perf_counter()
    return time_after - time_before, output

In [13]:
for i in range(5):
    t_, s_ = timeFunction(retrieveQueryAndGetScore, **{"model": models['TF-IDF'], "query": queries[i], "k": queries[i].GetNumberOfRelevantDocuments()})



QueryToVector Elapsed: 0.0019011497497558594s
CalculateScores Elapsed: 0.03876900672912598s
QueryToVector Elapsed: 0.0010249614715576172s
CalculateScores Elapsed: 0.02147984504699707s
QueryToVector Elapsed: 0.0015799999237060547s
CalculateScores Elapsed: 0.03039836883544922s
QueryToVector Elapsed: 0.0009589195251464844s
CalculateScores Elapsed: 0.023617982864379883s
QueryToVector Elapsed: 0.0007722377777099609s
CalculateScores Elapsed: 0.018088817596435547s
